In [1]:
# 90 days back up to 2019-04-06
import pandas as pd
import datetime
import os
import numpy as np
import hashlib
import gc
os.getcwd()

'/home/jian/celery/Google_LiveRamp'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            


thismonday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
thismonday=datetime.date(2019,4,8)
print("thismonday", thismonday)

last_week_end_saturday=thismonday-datetime.timedelta(days=2)

print("last_week_end_saturday: ",last_week_end_saturday)

start_90_days=last_week_end_saturday-datetime.timedelta(days=89)
print("start_90_days: ",start_90_days)

thismonday 2019-04-08
last_week_end_saturday:  2019-04-06
start_90_days:  2019-01-07


In [3]:
writer_pather="/home/jian/celery/Google_LiveRamp/output/"
data_root_2019="/home/jian/BigLots/2019_by_weeks/"

In [4]:
files_daily_list=[x for x in list(recursive_file_gen(data_root_2019)) if "Daily" in x]

sub_class_level_daily_df=pd.DataFrame({"file_path":files_daily_list})
sub_class_level_daily_df['week_end_dt']=sub_class_level_daily_df['file_path'].apply(lambda x: x.split("_weeks/MediaStorm_")[1][:10])
sub_class_level_daily_df=sub_class_level_daily_df[(sub_class_level_daily_df['week_end_dt']>=str(start_90_days)) & (sub_class_level_daily_df['week_end_dt']<="2019-02-09")]
sub_class_level_daily_df=sub_class_level_daily_df.sort_values("week_end_dt")


item_level_daily_df=pd.DataFrame({"file_path":files_daily_list})
item_level_daily_df['week_end_dt']=item_level_daily_df['file_path'].apply(lambda x: x.split("_weeks/MediaStorm_")[1][:10])
item_level_daily_df=item_level_daily_df[(item_level_daily_df['week_end_dt']>="2019-02-16") & (item_level_daily_df['week_end_dt']<="2019-04-06")]
item_level_daily_df=item_level_daily_df.sort_values("week_end_dt")




In [5]:
i_counter=0
data_last_90_days=pd.DataFrame()
for file_sub_class in sub_class_level_daily_df['file_path'].tolist():
    df=pd.read_table(file_sub_class,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt','subclass_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]    
    df=df[df['location_id']!="6990"]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df=df.rename(columns={"transaction_dt":"transaction_timestamp","subclass_transaction_amt":"Conversion_Amount"})
    df['transaction_timestamp']=df['transaction_timestamp'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    data_last_90_days=data_last_90_days.append(df)
    i_counter+=1
    print(i_counter,datetime.datetime.now())

1 2019-04-15 15:38:21.834860
2 2019-04-15 15:39:10.185455
3 2019-04-15 15:39:55.961365
4 2019-04-15 15:40:34.520565
5 2019-04-15 15:41:14.435255


In [6]:

for file_item_level in item_level_daily_df['file_path'].tolist():
    df=pd.read_table(file_item_level,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df[df['location_id']!="6990"]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df.rename(columns={"transaction_dt":"transaction_timestamp","item_transaction_amt":"Conversion_Amount"})
    df['transaction_timestamp']=df['transaction_timestamp'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    data_last_90_days=data_last_90_days.append(df)
    i_counter+=1
    print(i_counter,datetime.datetime.now())
data_last_90_days=data_last_90_days[data_last_90_days['transaction_timestamp']>=datetime.date(int(str(start_90_days).split("-")[0]),int(str(start_90_days).split("-")[1]),int(str(start_90_days).split("-")[2]))]


6 2019-04-15 15:42:01.069579
7 2019-04-15 15:42:47.735543
8 2019-04-15 15:43:40.081591
9 2019-04-15 15:44:32.096346
10 2019-04-15 15:45:22.816569
11 2019-04-15 15:46:14.011225
12 2019-04-15 15:47:05.124093
13 2019-04-15 15:48:10.756140


In [7]:
Master_2019_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aster" in x) & (".txt" in x) ]
Master_2018_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aster" in x) & (".txt" in x) ]


weekly_df=pd.DataFrame({"file_path":Master_2019_weekly+Master_2018_weekly})
weekly_df['date']=weekly_df['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10])
weekly_df['date']=weekly_df['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
weekly_df=weekly_df.sort_values("date",ascending=False).reset_index()
del weekly_df['index']
weekly_df.head(5)

# No need for the recent posibble file foloder

,file_path,date
0,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-04-06
1,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-30
2,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-23
3,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-16
4,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-09


In [8]:
data_0=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip",
                     dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code']).drop_duplicates()
print(data_0.shape)


(1873409, 3)


In [9]:
data_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStormCustTot-hashed-email.txt",
                     dtype=str,header=None,usecols=[0,1,5])
data_1.columns=['customer_id_hashed','email_address_hash','customer_zip_code']
data_1['customer_id_hashed']=data_1['customer_id_hashed'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())
print(data_1.shape)

(19337597, 3)


In [10]:
data_2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'MediaStormCustomerTransactionTotals_2018-01-09_2018-03-31.txt',
                     sep = ',',dtype = str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_2.shape)
data_3 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'Existing Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_3.shape)
data_4 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'New Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_4.shape)

(7255226, 3)
(5759051, 3)
(659127, 3)


In [11]:
master_old=data_4.append(data_3).append(data_2).append(data_1).append(data_0).drop_duplicates()
del data_4
del data_3
del data_2
del data_1
del data_0
gc.collect()

49

In [12]:
all_weekly_biweekly_master_file=pd.DataFrame()
for file_path in weekly_df['file_path'].tolist():
    df=pd.read_table(file_path,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|")
    all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(df)

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(master_old)
print(all_weekly_biweekly_master_file.shape)
print(len(all_weekly_biweekly_master_file['customer_id_hashed'].unique()))

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.drop_duplicates('customer_id_hashed')
print(all_weekly_biweekly_master_file.shape)

(25122556, 3)
25087795
(25087795, 3)


In [13]:
print(data_last_90_days.shape)
print(data_last_90_days[['customer_id_hashed','transaction_timestamp']].drop_duplicates().shape)

data_last_90_days.head(2)

(16463959, 3)
(16463959, 2)


,customer_id_hashed,transaction_timestamp,Conversion_Amount
0,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,2019-01-10,7.95
1,00002227290330cf6112d04470e512057ba4b50ab64c9f...,2019-01-10,27.23


In [14]:
data_last_90_days=pd.merge(data_last_90_days,all_weekly_biweekly_master_file,on="customer_id_hashed",how="left").rename(columns={"email_address_hash":"Email_1","customer_zip_code":"Zip_Code"})
data_last_90_days.head(2)

total_rows_with_na_str=str(data_last_90_days.shape[0])
print("Total_Raws: "+total_rows_with_na_str)
na_emails_rows_count_str=str(data_last_90_days[pd.isnull(data_last_90_days['Email_1'])].shape[0])
print("Null Email rows excluded: "+na_emails_rows_count_str)
data_last_90_days=data_last_90_days[~pd.isnull(data_last_90_days['Email_1'])]
del data_last_90_days['customer_id_hashed']

print("Na_Email_Rows_Pctg: ",int(na_emails_rows_count_str)/int(total_rows_with_na_str))

Total_Raws: 16463959
Null Email rows excluded: 1017500
Na_Email_Rows_Pctg:  0.06180166022036376


In [15]:
data_last_90_days.head(2)

,transaction_timestamp,Conversion_Amount,Email_1,Zip_Code
0,2019-01-10,7.95,787ae23b29bbc6fe721c28c14d0227575c534dca7470e9...,43612
1,2019-01-10,27.23,216b81a8a9eee53938c376a96eaace39016f7787626786...,77840


In [16]:
data_last_90_days=data_last_90_days[["Email_1","Zip_Code","transaction_timestamp", "Conversion_Amount"]].rename(columns={"Conversion_Amount":"transaction_amount"})
data_last_90_days['transaction_amount']=data_last_90_days['transaction_amount'].apply(lambda x: np.round(x,2)).astype(str)
data_last_90_days['transaction_amount']=data_last_90_days['transaction_amount'].apply(lambda x: x.split(".")[0]+"."+x.split(".")[1].ljust(2,"0"))
data_last_90_days['transaction_category']="In_Store"

data_last_90_days['Zip_Code']="00000"

data_last_90_days=data_last_90_days[['Zip_Code','Email_1','transaction_category','transaction_timestamp','transaction_amount']]

In [17]:
data_max_date=data_last_90_days['transaction_timestamp'].max()
data_max_date

datetime.date(2019, 4, 6)

In [18]:
data_min_date=data_last_90_days['transaction_timestamp'].min()
data_min_date

datetime.date(2019, 1, 7)

In [19]:
data_last_90_days.shape

(15446459, 5)

In [20]:
data_max_date-data_min_date

datetime.timedelta(89)

In [21]:
len(data_last_90_days['transaction_timestamp'].unique())

90

In [22]:
writer_pather

'/home/jian/celery/Google_LiveRamp/output/'

In [23]:
local_path=writer_pather+"BL_LR_GoogleStoreSales_90days_"+str(data_min_date)+"_"+str(data_max_date)+"_JL_"+str(datetime.datetime.now().date())+".txt"

data_last_90_days.to_csv(local_path,index=False,sep="|")


In [24]:
data_last_90_days['transaction_amount'].astype(float).sum()

697681124.07000029

In [25]:
data_last_90_days.head(5)

,Zip_Code,Email_1,transaction_category,transaction_timestamp,transaction_amount
0,00000,787ae23b29bbc6fe721c28c14d0227575c534dca7470e9...,In_Store,2019-01-10,7.95
1,00000,216b81a8a9eee53938c376a96eaace39016f7787626786...,In_Store,2019-01-10,27.23
2,00000,884c7d27f0df2fdd99383fbe9ca4515281160dd08fb351...,In_Store,2019-01-07,9.90
3,00000,3f029ca17f7d6d7c711c18b63ebd245798a8efd2b386d8...,In_Store,2019-01-12,3.00
4,00000,4b86884f30bce7a304559a5acd1a1bbdba1daaa151267f...,In_Store,2019-01-10,26.50


In [26]:
local_path=="/home/jian/celery/Google_LiveRamp/output/BL_LR_GoogleStoreSales_90days_2019-01-07_2019-04-06_JL_2019-04-15.txt"

True

In [27]:
data_last_90_days['transaction_amount'].astype(float).sum()/1200000000

0.58140093672500026

In [28]:
data_last_90_days.columns.tolist()==['Zip_Code','Email_1','transaction_category','transaction_timestamp','transaction_amount']



True

In [30]:
import paramiko

host = "sftp.liveramp.com" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "Jubaplus2019!" #hard-coded
username = "big-lots-ga-aw" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [31]:
# local_path defined above before saving the local txt
remote_path="/uploads/"+os.path.basename(local_path)
sftp.put(local_path,remote_path)
sftp.close()